In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 
import random

In [2]:
service = Service(executable_path='chromedriver.exe')
driver = webdriver.Chrome(service=service)

In [3]:
url = 'https://companiesmarketcap.com/automakers/largest-automakers-by-market-cap/'

In [4]:
driver.get(url)
WebDriverWait(driver, 20).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, 'table.marketcap-table'))
)

<selenium.webdriver.remote.webelement.WebElement (session="c1602dc7bdf1180298197ddbec99e82e", element="f.5385001EC280B18ED7CD383A93DB794A.d.D35B31C3302561A11CAEEE48B54C7745.e.125")>

In [5]:
table = driver.find_element(By.CSS_SELECTOR, "table.marketcap-table")

In [6]:
print(table)

<selenium.webdriver.remote.webelement.WebElement (session="c1602dc7bdf1180298197ddbec99e82e", element="f.5385001EC280B18ED7CD383A93DB794A.d.D35B31C3302561A11CAEEE48B54C7745.e.125")>


In [7]:
import pandas as pd

In [8]:
thead = table.find_element(By.TAG_NAME, 'thead')
column_names = thead.find_elements(By.TAG_NAME, 'th')

In [9]:
headers = [name.text for name in column_names]
final_headers = [headers[i] for i in [2,3,4,5,7]]

In [10]:
print(final_headers)

['Name', 'M. Cap', 'Price', '1d', 'C.']


In [11]:
body = table.find_element(By.TAG_NAME, 'tbody')

In [12]:
df = pd.DataFrame(columns=final_headers)

In [13]:
df.head()

,Name,M. Cap,Price,1d,C.


In [14]:
rows = body.find_elements(By.TAG_NAME, 'tr')

In [15]:
def moneyToFloat(val):
    if isinstance(val, str):
        val = val.replace('$', '').replace(',', '').strip()
        if 'T' in val or "Trillion" in val:
            return float(val.replace('T', '').replace("rillion", '')) * 1e12
        elif 'B' in val or "Billion" in val:
            return float(val.replace('B', '').replace("illion", '')) * 1e9
        elif 'M' in val or "Million" in val:
            return float(val.replace('M', '').replace("illion", '')) * 1e6

        try:
            return float(val)
        except:
            return None

In [16]:
def decimalToFloat(val):
    if isinstance(val, str):
        return float(val.replace('%', '').replace(',', '').strip()) / 100
    return val

In [17]:
for row in rows:
    tds = row.find_elements(By.TAG_NAME, 'td')
    if len(tds) >= 7:
        row_data = [tds[i].text.strip() for i in [2,3,4,5]]
        cleaned_name = tds[2].text.strip().split('\n')[0].strip()
        row_data[0] = cleaned_name
        arrow = tds[5].find_element(By.TAG_NAME, 'span')
        one_day_text = tds[5].text
        one_day = decimalToFloat(one_day_text)
        if "percentage-red" in arrow.get_attribute('class'):
            one_day *= -1
            row_data[-1] = one_day
        else:
            row_data[-1] = one_day
        country_span = tds[-1].find_element(By.CSS_SELECTOR, 'span.responsive-hidden')
        country_name = country_span.get_attribute('innerHTML').strip()
        row_data.append(country_name)
        df.loc[len(df)] = row_data

In [18]:
df.tail()

,Name,M. Cap,Price,1d,C.
57,REE Automotive,$21.06 M,$0.75,-0.0558,Israel
58,Phoenix Motor,$10.57 M,$0.23,0.0000,USA
59,Canoo,$5.35 M,$0.37,0.0000,USA
60,Mullen Automotive,$4.39 M,$8.11,-0.4963,USA
61,Arcimoto,$4.24 M,$0.38,-0.0594,USA


In [19]:
link_div = driver.find_element(By.CSS_SELECTOR, "div.ranking-bar")
links = link_div.find_elements(By.TAG_NAME, 'a')
other_metrics = [link.get_attribute('href') for link in links]

In [20]:
print(other_metrics)

['https://companiesmarketcap.com/automakers/largest-automakers-by-market-cap/', 'https://companiesmarketcap.com/automakers/most-profitable-automakers/', 'https://companiesmarketcap.com/automakers/largest-automakers-by-revenue/', 'https://companiesmarketcap.com/automakers/automakers-ranked-by-pe-ratio/', 'https://companiesmarketcap.com/automakers/automakers-ranked-by-dividend-yield/', 'https://companiesmarketcap.com/automakers/automakers-ranked-by-operating-margin/', 'https://companiesmarketcap.com/automakers/largest-automakers-by-number-of-employees/', 'https://companiesmarketcap.com/automakers/top-automakers-by-total-assets/', 'https://companiesmarketcap.com/automakers/top-automakers-by-net-assets/', 'https://companiesmarketcap.com/automakers/automakers-with-the-highest-liabilities/', 'https://companiesmarketcap.com/automakers/automakers-with-the-highest-debt/', 'https://companiesmarketcap.com/automakers/automakers-with-the-most-cash-on-hand/', 'https://companiesmarketcap.com/automake

In [21]:
metrics = [metric for metric in other_metrics[1:-1]]

In [22]:
print(metrics)

['https://companiesmarketcap.com/automakers/most-profitable-automakers/', 'https://companiesmarketcap.com/automakers/largest-automakers-by-revenue/', 'https://companiesmarketcap.com/automakers/automakers-ranked-by-pe-ratio/', 'https://companiesmarketcap.com/automakers/automakers-ranked-by-dividend-yield/', 'https://companiesmarketcap.com/automakers/automakers-ranked-by-operating-margin/', 'https://companiesmarketcap.com/automakers/largest-automakers-by-number-of-employees/', 'https://companiesmarketcap.com/automakers/top-automakers-by-total-assets/', 'https://companiesmarketcap.com/automakers/top-automakers-by-net-assets/', 'https://companiesmarketcap.com/automakers/automakers-with-the-highest-liabilities/', 'https://companiesmarketcap.com/automakers/automakers-with-the-highest-debt/', 'https://companiesmarketcap.com/automakers/automakers-with-the-most-cash-on-hand/']


In [23]:
for metric in metrics:
    driver.get(metric)
    time.sleep(random.uniform(2, 8))
    WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "div.ranking-bar"))
    )
    link_div = driver.find_element(By.CSS_SELECTOR, "div.ranking-bar")
    links = link_div.find_elements(By.TAG_NAME, 'a')
    metric_name = None
    for link in links:
        if link.get_attribute('href') in metric:
            metric_name = link.text.strip()
            print(metric_name)
    df[metric_name] = None
    table = driver.find_element(By.CSS_SELECTOR, 'table.marketcap-table')
    body = table.find_element(By.TAG_NAME, 'tbody')
    rows = body.find_elements(By.TAG_NAME, 'tr')
    for row in rows:
        tds = row.find_elements(By.TAG_NAME, 'td')
        if len(tds) >= 4:
            company_name = tds[2].text.strip()
            cleaned_name = company_name.split('\n')[0].strip()
            metric_value = tds[3].text.strip()
            df.loc[df['Name'] == cleaned_name, metric_name] = metric_value

Earnings
Revenue
P/E ratio
Dividend %
Operating Margin
Employees
Total assets
Net assets
Total liabilities
Total debt
Cash on hand


In [24]:
df.head(25)

,Name,M. Cap,Price,1d,C.,Earnings,Revenue,P/E ratio,Dividend %,Operating Margin,Employees,Total assets,Net assets,Total liabilities,Total debt,Cash on hand
0,Tesla,$917.00 B,$284.70,-0.1426,USA,$8.39 B,$95.72 B,143,0.00%,8.76%,"140,473",$125.11 B,$75.41 B,$49.69 B,$13.12 B,$36.99 B
1,Toyota,$241.99 B,$184.40,-0.0189,Japan,$46.41 B,$305.78 B,7.01,1.41%,15.44%,"384,158",$603.04 B,$234.76 B,$368.27 B,$252.10 B,$88.41 B
2,Xiaomi,$179.23 B,$6.92,0.0082,China,$3.38 B,$41.78 B,61.9,0.00%,7.68%,"33,627",$47.09 B,$23.83 B,$23.25 B,$4.72 B,$13.47 B
3,BYD,$155.22 B,$50.09,-0.0022,China,$7.94 B,$114.05 B,55.8,0.86%,6.96%,"968,900",$115.73 B,$33.89 B,$81.83 B,$5.66 B,$21.12 B
4,Ferrari,$85.60 B,$480.36,-0.0012,Italy,$2.13 B,$7.41 B,51.2,0.00%,28.73%,"5,290",$10.72 B,$3.86 B,$6.86 B,$3.60 B,$2.08 B
5,Mercedes-Benz,$56.51 B,$58.69,0.0006,Germany,$14.15 B,$153.92 B,5.78,8.37%,9.20%,"166,056",$282.79 B,$103.19 B,$179.60 B,$93.94 B,$27.18 B
6,BMW,$54.09 B,$87.44,-0.0016,Germany,$11.25 B,$150.23 B,6.45,0.00%,7.51%,"154,950",$287.25 B,$104.44 B,$182.81 B,$88.97 B,$23.69 B
7,Volkswagen,$53.29 B,$105.30,-0.0073,Germany,$15.79 B,$351.87 B,5.18,6.90%,4.50%,"656,134",$692.04 B,$217.70 B,$474.34 B,$206.07 B,$69.99 B
8,Maruti Suzuki India,$45.73 B,$145.48,0.0277,India,$2.34 B,$17.14 B,27.9,1.01%,13.65%,"16,259",$15.42 B,$11.25 B,$4.17 B,$10.17 M,$1.43 B
9,General Motors,$45.42 B,$47.25,-0.0088,USA,$8.37 B,$188.44 B,4.20,0.76%,4.44%,"163,000",$282.10 B,$66.42 B,$215.67 B,$132.67 B,$27.48 B


In [25]:
df.tail(25)

,Name,M. Cap,Price,1d,C.,Earnings,Revenue,P/E ratio,Dividend %,Operating Margin,Employees,Total assets,Net assets,Total liabilities,Total debt,Cash on hand
37,Ford Otosan,$7.84 B,$2.24,-0.0045,Turkey,$1.30 B,$17.03 B,0.5443,3.58%,7.67%,"23,934",$9.97 B,$3.28 B,$6.68 B,$3.65 B,$1.30 B
38,Zeekr,$6.78 B,$26.68,-0.0045,China,-$0.59 Billion,$11.52 B,-9.16,0.00%,-5.06%,"15,859",$8.41 B,$-3.44 Billion,$11.85 B,$5.06 B,$1.03 B
39,Lucid Motors,$6.49 B,$2.13,-0.0448,USA,-$2.37 Billion,$0.87 B,-1.76,0.00%,-271.48%,"6,500",$9.21 B,$3.18 B,$6.03 B,$2.55 B,$3.61 B
40,Volvo Car,$5.34 B,$1.80,-0.0097,Sweden,$1.82 B,$37.26 B,4.23,0.00%,4.90%,"42,000",$37.80 B,$14.96 B,$22.83 B,$3.28 B,$4.71 B
41,FAW Car,$4.78 B,$0.97,-0.0014,China,$20.4 M,$7.46 B,41.2,2.14%,0.29%,"20,344",$10.88 B,$3.66 B,$7.21 B,$2.64 M,$2.77 B
42,Mitsubishi Motors,$3.86 B,$2.89,0.0073,Japan,$1.23 B,$19.03 B,-4.43,3.65%,6.48%,"28,982",$16.21 B,$6.89 B,$9.31 B,$3.25 B,$4.45 B
43,Mazda,$3.68 B,$5.84,0.0155,Japan,$2.08 B,$32.92 B,2.89,6.60%,6.34%,"48,685",$25.04 B,$11.60 B,$13.43 B,$3.80 B,$6.09 B
44,Dongfeng Motor,$3.61 B,$0.44,-0.0471,China,-$0.16 Billion,$14.55 B,-6.60,1.59%,0.00%,"112,706",$44.53 B,$21.21 B,$23.32 B,$7.79 B,$13.23 B
45,Tofaş Türk Otomobil Fabrikası,$2.33 B,$4.66,-0.0103,Turkey,$93.9 M,$3.19 B,14.2,6.57%,2.89%,"5,814",$2.75 B,$1.18 B,$1.57 B,$0.78 B,$0.67 B
46,Polaris,$2.29 B,$40.82,-0.0131,USA,$0.20 B,$6.97 B,56.7,4.88%,1.96%,"18,500",$5.45 B,$1.23 B,$4.21 B,$2.18 B,$0.29 B


In [26]:
#df.to_csv('financials.csv', index=False)

In [27]:
df.columns = [col.strip().replace('.', '').replace(' ', '_') for col in df.columns]

In [28]:
df.head()

,Name,M_Cap,Price,1d,C,Earnings,Revenue,P/E_ratio,Dividend_%,Operating_Margin,Employees,Total_assets,Net_assets,Total_liabilities,Total_debt,Cash_on_hand
0,Tesla,$917.00 B,$284.70,-0.1426,USA,$8.39 B,$95.72 B,143,0.00%,8.76%,"140,473",$125.11 B,$75.41 B,$49.69 B,$13.12 B,$36.99 B
1,Toyota,$241.99 B,$184.40,-0.0189,Japan,$46.41 B,$305.78 B,7.01,1.41%,15.44%,"384,158",$603.04 B,$234.76 B,$368.27 B,$252.10 B,$88.41 B
2,Xiaomi,$179.23 B,$6.92,0.0082,China,$3.38 B,$41.78 B,61.9,0.00%,7.68%,"33,627",$47.09 B,$23.83 B,$23.25 B,$4.72 B,$13.47 B
3,BYD,$155.22 B,$50.09,-0.0022,China,$7.94 B,$114.05 B,55.8,0.86%,6.96%,"968,900",$115.73 B,$33.89 B,$81.83 B,$5.66 B,$21.12 B
4,Ferrari,$85.60 B,$480.36,-0.0012,Italy,$2.13 B,$7.41 B,51.2,0.00%,28.73%,"5,290",$10.72 B,$3.86 B,$6.86 B,$3.60 B,$2.08 B


In [29]:
df.columns.values[4] = 'Country'

In [30]:
df.head()

,Name,M_Cap,Price,1d,Country,Earnings,Revenue,P/E_ratio,Dividend_%,Operating_Margin,Employees,Total_assets,Net_assets,Total_liabilities,Total_debt,Cash_on_hand
0,Tesla,$917.00 B,$284.70,-0.1426,USA,$8.39 B,$95.72 B,143,0.00%,8.76%,"140,473",$125.11 B,$75.41 B,$49.69 B,$13.12 B,$36.99 B
1,Toyota,$241.99 B,$184.40,-0.0189,Japan,$46.41 B,$305.78 B,7.01,1.41%,15.44%,"384,158",$603.04 B,$234.76 B,$368.27 B,$252.10 B,$88.41 B
2,Xiaomi,$179.23 B,$6.92,0.0082,China,$3.38 B,$41.78 B,61.9,0.00%,7.68%,"33,627",$47.09 B,$23.83 B,$23.25 B,$4.72 B,$13.47 B
3,BYD,$155.22 B,$50.09,-0.0022,China,$7.94 B,$114.05 B,55.8,0.86%,6.96%,"968,900",$115.73 B,$33.89 B,$81.83 B,$5.66 B,$21.12 B
4,Ferrari,$85.60 B,$480.36,-0.0012,Italy,$2.13 B,$7.41 B,51.2,0.00%,28.73%,"5,290",$10.72 B,$3.86 B,$6.86 B,$3.60 B,$2.08 B


In [31]:
money_cols = ['M_Cap', 'Price', 'Earnings', 'Revenue', 'Total_assets',
              'Net_assets', 'Total_liabilities', 'Total_debt', 'Cash_on_hand']
df[money_cols] = df[money_cols].applymap(moneyToFloat)

C:\Users\PC\AppData\Local\Temp\ipykernel_11040\3295926965.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[money_cols] = df[money_cols].applymap(moneyToFloat)


In [32]:
df.head()

,Name,M_Cap,Price,1d,Country,Earnings,Revenue,P/E_ratio,Dividend_%,Operating_Margin,Employees,Total_assets,Net_assets,Total_liabilities,Total_debt,Cash_on_hand
0,Tesla,9.170000e+11,284.70,-0.1426,USA,8.390000e+09,9.572000e+10,143,0.00%,8.76%,"140,473",1.251100e+11,7.541000e+10,4.969000e+10,1.312000e+10,3.699000e+10
1,Toyota,2.419900e+11,184.40,-0.0189,Japan,4.641000e+10,3.057800e+11,7.01,1.41%,15.44%,"384,158",6.030400e+11,2.347600e+11,3.682700e+11,2.521000e+11,8.841000e+10
2,Xiaomi,1.792300e+11,6.92,0.0082,China,3.380000e+09,4.178000e+10,61.9,0.00%,7.68%,"33,627",4.709000e+10,2.383000e+10,2.325000e+10,4.720000e+09,1.347000e+10
3,BYD,1.552200e+11,50.09,-0.0022,China,7.940000e+09,1.140500e+11,55.8,0.86%,6.96%,"968,900",1.157300e+11,3.389000e+10,8.183000e+10,5.660000e+09,2.112000e+10
4,Ferrari,8.560000e+10,480.36,-0.0012,Italy,2.130000e+09,7.410000e+09,51.2,0.00%,28.73%,"5,290",1.072000e+10,3.860000e+09,6.860000e+09,3.600000e+09,2.080000e+09


In [33]:
df['Dividend_%'] = df['Dividend_%'].apply(decimalToFloat)
df['Operating_Margin'] = df['Operating_Margin'].apply(decimalToFloat)

In [34]:
df.head()

,Name,M_Cap,Price,1d,Country,Earnings,Revenue,P/E_ratio,Dividend_%,Operating_Margin,Employees,Total_assets,Net_assets,Total_liabilities,Total_debt,Cash_on_hand
0,Tesla,9.170000e+11,284.70,-0.1426,USA,8.390000e+09,9.572000e+10,143,0.0000,0.0876,"140,473",1.251100e+11,7.541000e+10,4.969000e+10,1.312000e+10,3.699000e+10
1,Toyota,2.419900e+11,184.40,-0.0189,Japan,4.641000e+10,3.057800e+11,7.01,0.0141,0.1544,"384,158",6.030400e+11,2.347600e+11,3.682700e+11,2.521000e+11,8.841000e+10
2,Xiaomi,1.792300e+11,6.92,0.0082,China,3.380000e+09,4.178000e+10,61.9,0.0000,0.0768,"33,627",4.709000e+10,2.383000e+10,2.325000e+10,4.720000e+09,1.347000e+10
3,BYD,1.552200e+11,50.09,-0.0022,China,7.940000e+09,1.140500e+11,55.8,0.0086,0.0696,"968,900",1.157300e+11,3.389000e+10,8.183000e+10,5.660000e+09,2.112000e+10
4,Ferrari,8.560000e+10,480.36,-0.0012,Italy,2.130000e+09,7.410000e+09,51.2,0.0000,0.2873,"5,290",1.072000e+10,3.860000e+09,6.860000e+09,3.600000e+09,2.080000e+09


In [35]:
def removeCommas(val):
    if isinstance(val, str):
        if val == "N/A" or val == "":
            return None
        val = val.replace(',', '').strip()
        try:
            return int(float(val))
        except ValueError:
            return None         
    return val

In [36]:
df['Employees'] = df['Employees'].apply(removeCommas)
df.head()

,Name,M_Cap,Price,1d,Country,Earnings,Revenue,P/E_ratio,Dividend_%,Operating_Margin,Employees,Total_assets,Net_assets,Total_liabilities,Total_debt,Cash_on_hand
0,Tesla,9.170000e+11,284.70,-0.1426,USA,8.390000e+09,9.572000e+10,143,0.0000,0.0876,140473.0,1.251100e+11,7.541000e+10,4.969000e+10,1.312000e+10,3.699000e+10
1,Toyota,2.419900e+11,184.40,-0.0189,Japan,4.641000e+10,3.057800e+11,7.01,0.0141,0.1544,384158.0,6.030400e+11,2.347600e+11,3.682700e+11,2.521000e+11,8.841000e+10
2,Xiaomi,1.792300e+11,6.92,0.0082,China,3.380000e+09,4.178000e+10,61.9,0.0000,0.0768,33627.0,4.709000e+10,2.383000e+10,2.325000e+10,4.720000e+09,1.347000e+10
3,BYD,1.552200e+11,50.09,-0.0022,China,7.940000e+09,1.140500e+11,55.8,0.0086,0.0696,968900.0,1.157300e+11,3.389000e+10,8.183000e+10,5.660000e+09,2.112000e+10
4,Ferrari,8.560000e+10,480.36,-0.0012,Italy,2.130000e+09,7.410000e+09,51.2,0.0000,0.2873,5290.0,1.072000e+10,3.860000e+09,6.860000e+09,3.600000e+09,2.080000e+09


In [37]:
df.to_csv('Final2_Financials.csv', index=False)